In [ ]:
import numpyro
from numpyro import sample, handlers, plate
import numpyro.distributions as dist
from jax import random
from numpyro.infer import MCMC, NUTS
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

In [ ]:
## test multiple observations from single latent

def generative_process(mu):
    # mu = sample('mu', dist.Normal(0, 1))
    with plate('i', 5):
        obs = sample('obs', dist.Normal(mu, 0.2))
    return obs

def latent_variable_model(obs):
    mu = sample('mu', dist.Normal(0, 1))
    with plate('i', 5):
        obs = sample('obs', dist.Normal(mu, 0.2), obs=out)
    return obs

with handlers.seed(rng_seed=2):
    out = generative_process(1.4)
    
out

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out)

# s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

In [ ]:
## test inferring latent variables from observation of differences
## linear normal model

def generative_process(mu):
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Normal(mu[i] - mu[j], 0.1))
            diffs.append(diff)
    return diffs

test_ranking = [0, 0.1, 0.2, 0.9]
with handlers.seed(rng_seed=2):
    out = generative_process(test_ranking)
    
for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        pass#print(test_ranking[i] - test_ranking[j])

out

def latent_variable_model(obs):
    with plate('i', 4):
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            sample(f'diff_{i}{j}', dist.Normal(mu[i] - mu[j], 0.1), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(est_mu[i] - est_mu[j])

out

In [ ]:
## test inferring latent variables from observation of differences
## bernoulli observation model

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

random_key = 0

def generative_process(mu):
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Bernoulli(ndtr(mu[i] - mu[j])))
            diffs.append(diff)
    return diffs

test_ranking = jnp.array([0, 0.9, 0.2, 0.5]) * 2
random_key += 1
with handlers.seed(rng_seed=random_key):
    out = generative_process(test_ranking)
    
for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(test_ranking[i] - test_ranking[j])

out = jnp.array(out)
out

def latent_variable_model(obs):
    with plate('i', 4):
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            sample(f'diff_{i}{j}', dist.Bernoulli(ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(ndtr(est_mu[i] - est_mu[j]))

out

est_mu

In [ ]:
## test inferring latent variables from observation of differences
## binomial observation model

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

random_key = 0

def generative_process(mu, N=10):
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])))
            diffs.append(diff)
    return diffs

test_ranking = jnp.array([0, 0.9, 0.2, 0.5]) * 2
random_key += 1
with handlers.seed(rng_seed=random_key):
    out = generative_process(test_ranking, 100)
    
for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(test_ranking[i] - test_ranking[j])

out = jnp.array(out)
out

def latent_variable_model(obs, N=10):
    with plate('i', 4):
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out, 100)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(ndtr(est_mu[i] - est_mu[j]))

test_ranking, (est_mu - est_mu.min()) / (est_mu - est_mu.min()).max() * test_ranking.max()

In [ ]:
## test inferring latent variables from observation of differences
## binomial observation model
## set fixpoint -> 

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

random_key = 0

def generative_process(mu, N=10):
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])))
            diffs.append(diff)
    return diffs

test_ranking = jnp.array([0, 0.9, 0.2, 0.5]) * 2
random_key += 1
with handlers.seed(rng_seed=random_key):
    out = generative_process(test_ranking, 100)
    
for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(test_ranking[i] - test_ranking[j])

out = jnp.array(out)
out

def latent_variable_model(obs, N=10):
    mu_0 = sample('mu_0', dist.Normal(0, 1), obs=0)
    with plate('i', 3) as i:
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)+1):
        for j in range(i+1, len(mu)+1):
            mu_i = mu[i-1] if i > 0 else mu_0
            mu_j = mu[j-1] if j > 0 else mu_0
            sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu_i - mu_j)), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out, 100)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

c = 0
for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        mu_i = est_mu[i-1] if i > 0 else 0
        mu_j = est_mu[j-1] if j > 0 else 0
        print(ndtr(mu_i - mu_j), out[c]/100)
        c += 1

test_ranking, est_mu * test_ranking.max() / est_mu.max(), out / 100

In [ ]:
## test latent variables generated by parabel from observation of differences
## binomial observation model

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

random_key = 0

def parabel(wr, a, b, c):
    return a + b * wr + c * wr ** 2

# next: include f(a, b, c) -> mu in generative process
def generative_process(wr, N=10):
    mu = parabel(wr, a, b, c)
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])))
            diffs.append(diff)
    return diffs

# parabel variables
a, b, c = -335.25752558,  1410.59741048, -1483.63084321#-239.66860676,  1050.59254591, -1147.81367583
test_input = jnp.array([0.43      , 0.4422449 , 0.4544898 , 0.46673469, 0.47897959])
test_ranking = jnp.array([-3.02398198, -1.59735369, -0.61562928, -0.07880874,  0.01310792])

# parabel variables
a, b, c = 1, 0, -1

test_input = jnp.array([-1, -0.5, 0, 0.5, 1])
test_ranking = parabel(test_input, a, b, c)

# plt.plot(test_input, test_ranking)

N_comparisons = 1000
random_key += 1
with handlers.seed(rng_seed=random_key):
    out = generative_process(test_input, N_comparisons)
    
# for i in range(len(test_ranking)):
#     for j in range(i+1, len(test_ranking)):
#         print(test_ranking[i] - test_ranking[j])

out = jnp.array(out)
out

def latent_variable_model(obs, N=10):
    with plate('i', len(test_ranking)):
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, out, N_comparisons)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(ndtr(est_mu[i] - est_mu[j]))

test_ranking - test_ranking.min(), (est_mu - est_mu.min()) / (est_mu - est_mu.min()).max() * (test_ranking - test_ranking.min()).max()

plt.plot(test_input, test_ranking)
plt.plot(test_input, (est_mu - est_mu.min()) / (est_mu - est_mu.min()).max() * (test_ranking - test_ranking.min()).max())
mu = s['mu']
for i in range(100):
    mu_sampled = mu[-i]
    y = (mu_sampled - mu_sampled.min()) / (mu_sampled - mu_sampled.min()).max() * (test_ranking - test_ranking.min()).max()
    plt.plot(test_input, y, 'x')

In [ ]:
## test inferring parabola parameters via latent variables from observation of differences
## binomial observation model

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

random_key = 0

# parabel variables
a, b, c = -335.25752558,  1410.59741048, -1483.63084321#-239.66860676,  1050.59254591, -1147.81367583
def parabel(wr, a, b, c):
    return a + b * wr + c * wr ** 2

# next: include f(a, b, c) -> mu in generative process
def generative_process(wr, N=10):
    mu = parabel(wr, a, b, c)
    diffs = []
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            diff = sample(f'diff_{i}{j}', dist.Binomial(N, ndtr(mu[i] - mu[j])))
            diffs.append(diff)
    return diffs

test_input = jnp.array([0.43      , 0.4422449 , 0.4544898 , 0.46673469, 0.47897959])
test_ranking = jnp.array([-3.02398198, -1.59735369, -0.61562928, -0.07880874,  0.01310792])
# parabel variables
a, b, c = 1, 0, -1
test_input = jnp.linspace(-1, 1, 10)
test_ranking = parabel(test_input, a, b, c)

random_key += 1
N_comparisons = 1000000
with handlers.seed(rng_seed=random_key):
    out = generative_process(test_input, N_comparisons)
    
# for i in range(len(test_ranking)):
#     for j in range(i+1, len(test_ranking)):
#         print(test_ranking[i] - test_ranking[j])

out = jnp.array(out)
out

def latent_variable_model(wr, obs, N_comparisons=10):
    # a = numpyro.param("a", 0)
    # b = numpyro.param("b", 0)
    # c = numpyro.param("c", 0)
    a = sample('a', dist.Normal(0, 10))
    b = sample('b', dist.Normal(0, 10))
    c = sample('c', dist.Normal(0, 10))
    #sigma = sample('sigma', dist.Uniform(0., 10.))
    sigma = 0.01
    N = len(test_ranking)
    # mu = [sample(f'mu_{i}', dist.Normal(parabel(wr, a, b, c), sigma)) for i in range(N)]
    mu = [sigma * sample(f'mu_{i}', dist.Normal(0, 1)) + parabel(wr[i], a, b, c)  for i in range(N)]
    #with plate('i', len(test_ranking)):
        #mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(N):
        for j in range(i+1, N):
            sample(f'diff_{i}{j}', dist.Binomial(N_comparisons, ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1

nuts_kernel = NUTS(latent_variable_model)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, test_input, out, N_comparisons)

s = mcmc.get_samples()
# plt.hist(s['mu'], bins=200)
# plt.show()

s['a'].mean(), s['b'].mean(), s['c'].mean()

In [ ]:
test_input
y

In [ ]:
plt.plot(test_input, test_ranking)
# plt.plot(test_input, (est_mu - est_mu.min()) / (est_mu - est_mu.min()).max() * (test_ranking - test_ranking.min()).max())
# mu = s['mu']
for i in range(100):
    mu_sampled = np.array([s[f'mu_{j}'][-i] for j in range(len(test_input))])
    y = (mu_sampled - mu_sampled.min()) / (mu_sampled - mu_sampled.min()).max() * (test_ranking - test_ranking.min()).max()
    plt.plot(test_input, y, 'x')

In [ ]:
est_mu = s['mu'].mean(axis=0)
# s['mu'].var(axis=0)

for i in range(len(test_ranking)):
    for j in range(i+1, len(test_ranking)):
        print(ndtr(est_mu[i] - est_mu[j]))

test_ranking - test_ranking.min(), (est_mu - est_mu.min()) / (est_mu - est_mu.min()).max() * (test_ranking - test_ranking.min()).max()